In [303]:
import pandas as pd
from tqdm import tqdm
import pickle
import numpy as np
import os

In [304]:
df = pickle.load(open("dataset_lvl2.pkl", "rb"))

In [305]:
df['match_id'] = df['match_id'].astype('int')

In [306]:
df['wickets_remaining'] = (10 - df.groupby(['match_id', 'batting_team'])['player_dismissed'].cumsum())

In [307]:
df['runs'] = df['batsman_runs'] + df['extras_runs']
df['current_score'] = df.groupby(['match_id', 'batting_team'])['runs'].cumsum()

In [308]:
groups = df.groupby(['match_id', 'batting_team'])

last_five = []

for group_key, group_data in groups:
    rolling_sum = group_data['runs'].rolling(window=30).sum().fillna(0)
    last_five.extend(rolling_sum.values.tolist())

df['last_five'] = last_five

In [309]:
df = df.groupby(['match_id', 'batting_team']).sum()['runs'].reset_index().merge(df, on=['match_id', 'batting_team'])

In [310]:
df['over'] = df['ball'].astype(int)
df['leftover'] = df['ball']-df['over']
df['balls_done'] = df['over']*6 + df['leftover']*10
df['balls_left'] = 120 - df['balls_done']
df['balls_left'] = df['balls_left'].astype(int)
df = df.drop('over',axis=1)
df = df.drop('leftover',axis=1)
df = df.drop('balls_done',axis=1)
df = df.drop('ball',axis=1)

In [311]:
df['city'] = df.apply(lambda row: row['venue'].split()[0] if pd.isnull(row['city']) else row['city'], axis=1)

In [312]:
df['crr'] = df['current_score'] / ((120 - df['balls_left']) / 6)

In [313]:
df.drop(['batsman_runs', 'extras_runs'], axis = 1, inplace=True)

In [314]:
df

,match_id,batting_team,runs_x,bowling_team,batsman,bowler,player_dismissed,city,venue,wickets_remaining,runs_y,current_score,last_five,balls_left,crr
0,3,India,98,West Indies,RG Sharma,O Thomas,0,Lauderhill,Central Broward Regional Park Stadium Turf Ground,10,0,0,0.0,119,0.000000
1,3,India,98,West Indies,RG Sharma,O Thomas,0,Lauderhill,Central Broward Regional Park Stadium Turf Ground,10,0,0,0.0,118,0.000000
2,3,India,98,West Indies,RG Sharma,O Thomas,0,Lauderhill,Central Broward Regional Park Stadium Turf Ground,10,1,1,0.0,117,2.000000
3,3,India,98,West Indies,S Dhawan,O Thomas,0,Lauderhill,Central Broward Regional Park Stadium Turf Ground,10,1,2,0.0,116,3.000000
4,3,India,98,West Indies,RG Sharma,O Thomas,0,Lauderhill,Central Broward Regional Park Stadium Turf Ground,10,0,2,0.0,115,2.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220053,1431,Pakistan,153,New Zealand,Shaheen Shah Afridi,BM Tickner,0,Auckland,Eden Park,2,1,149,0.0,4,7.706897
220054,1431,Pakistan,153,New Zealand,Faheem Ashraf,BM Tickner,1,Auckland,Eden Park,1,0,149,0.0,3,7.641026
220055,1431,Pakistan,153,New Zealand,Shaheen Shah Afridi,BM Tickner,0,Auckland,Eden Park,1,2,151,0.0,2,7.677966
220056,1431,Pakistan,153,New Zealand,Shaheen Shah Afridi,BM Tickner,0,Auckland,Eden Park,1,1,152,0.0,1,7.663866


In [315]:
df.rename(columns={'runs_x': 'final_score'}, inplace=True)

In [316]:
df = df[['batting_team', 'bowling_team', 'current_score', 'wickets_remaining', 'city', 'balls_left', 'last_five', 'final_score']]

In [317]:
required_teams = np.array(df['batting_team'].unique()[0:13])

type(np.array(df['batting_team']))

df = df[df['batting_team'].isin(required_teams) & df['bowling_team'].isin(required_teams)]

In [318]:
df.sample(df.shape[0])

,batting_team,bowling_team,current_score,wickets_remaining,city,balls_left,last_five,final_score
204138,West Indies,Pakistan,84,5,Mirpur,32,46.0,166
169592,England,South Africa,68,9,Durban,80,0.0,204
120339,West Indies,New Zealand,117,7,Lauderhill,28,46.0,177
157367,India,Zimbabwe,137,6,Harare,19,45.0,168
185626,India,Australia,114,2,Guwahati,4,39.0,118
...,...,...,...,...,...,...,...,...
156450,India,South Africa,167,4,Cape Town,3,56.0,172
76768,Pakistan,West Indies,42,9,Karachi,89,27.0,205
157466,Zimbabwe,India,97,7,Harare,45,38.0,170
43822,England,Pakistan,56,7,Dubai,53,0.0,130


In [319]:
pickle.dump(df, open("dataset_lvl3.pkl", 'wb'))